In [17]:
import pandas as pd
import numpy as np 
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [30]:
train_df = pd.read_csv('diamonds_data/train.csv')
test_df =  pd.read_csv('diamonds_data/test.csv')

In [3]:
y = train_df.price

In [11]:
train_df

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.50,Premium,F,VS2,61.5,58.0,7.32,7.34,4.51,9.588
1,1,2.01,Very Good,E,SI2,60.6,59.0,8.11,8.25,4.96,9.748
2,2,0.50,Ideal,E,SI1,61.6,57.0,5.13,5.09,3.15,7.255
3,3,0.25,Very Good,F,VVS2,61.6,57.0,4.05,4.08,2.50,6.450
4,4,0.52,Ideal,G,VS2,62.0,55.0,5.16,5.19,3.21,7.721
...,...,...,...,...,...,...,...,...,...,...,...
40450,40450,1.04,Very Good,I,SI2,59.6,57.0,6.60,6.62,3.94,8.190
40451,40451,0.51,Very Good,E,SI1,63.3,59.0,5.09,5.05,3.21,7.246
40452,40452,1.51,Ideal,H,VS2,62.6,57.0,7.37,7.33,4.60,9.277
40453,40453,2.02,Premium,H,VS2,61.3,60.0,8.16,8.11,4.99,9.680


In [12]:
encod = {
    "Fair":1,
    "Ideal":2,
    "Good":3,
    "Very Good":4,
    "Premium":5
}

In [13]:
train_df['cut'] = train_df['cut'].apply(lambda x: encod[x])

In [14]:
train_df = pd.get_dummies(train_df, columns=[ "color", "clarity"], drop_first=True)

In [15]:
train_df2= train_df.drop(['id', 'depth', 'table', 'z'], axis=1)

In [76]:
train_df2.head()

,carat,cut,x,y,price,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.50,5,7.32,7.34,9.588,0,1,0,0,0,0,0,0,0,0,1,0,0
1,2.01,4,8.11,8.25,9.748,1,0,0,0,0,0,0,0,1,0,0,0,0
2,0.50,2,5.13,5.09,7.255,1,0,0,0,0,0,0,1,0,0,0,0,0
3,0.25,4,4.05,4.08,6.450,0,1,0,0,0,0,0,0,0,0,0,0,1
4,0.52,2,5.16,5.19,7.721,0,0,1,0,0,0,0,0,0,0,1,0,0


In [79]:
X = train_df2.drop('price', axis=1)
y = train_df2.price

In [80]:
y

0        9.588
1        9.748
2        7.255
3        6.450
4        7.721
         ...  
40450    8.190
40451    7.246
40452    9.277
40453    9.680
40454    7.154
Name: price, Length: 40455, dtype: float64

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [82]:
from sklearn.model_selection import GridSearchCV
model = RandomForestRegressor()
params = {
    "n_estimators":[700,800], 
    "max_depth":[ None], 
    "min_samples_split":[9, 10, 11], 
    "min_samples_leaf":[1, 2]
}

In [83]:
df2_grid = GridSearchCV(estimator=model, param_grid=params, verbose=2, n_jobs=4, scoring= 'neg_mean_squared_error')

In [84]:
df2_grid.fit(X, y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=4,
             param_grid={'max_depth': [None], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [9, 10, 11],
                         'n_estimators': [700, 800]},
             scoring='neg_mean_squared_error', verbose=2)

In [85]:
df2_grid.best_params_

{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 9,
 'n_estimators': 700}

In [86]:
best = df2_grid.best_estimator_
best

RandomForestRegressor(min_samples_split=9, n_estimators=700)

In [87]:
y_pred_train = best.predict(X_train)
y_pred_test = best.predict(X_test)
mse_train = mean_squared_error(y_pred_train,y_train)
mse_test = mean_squared_error(y_pred_test,y_test)
mse_train, mse_test

(0.004329016939996856, 0.004185971566240093)

In [97]:
encod = {
    "Fair":1,
    "Ideal":2,
    "Good":3,
    "Very Good":4,
    "Premium":5
}

In [98]:
train_df['cut'] = train_df['cut'].apply(lambda x: encod[x])

In [99]:
train_df = pd.get_dummies(train_df, columns=[ "color", "clarity"], drop_first=True)

In [100]:
train_df2= train_df.drop(['id'], axis=1)

In [101]:
train_df2.head()

,carat,cut,depth,table,x,y,z,price,color_E,color_F,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.50,5,61.5,58.0,7.32,7.34,4.51,9.588,0,1,...,0,0,0,0,0,0,0,1,0,0
1,2.01,4,60.6,59.0,8.11,8.25,4.96,9.748,1,0,...,0,0,0,0,0,1,0,0,0,0
2,0.50,2,61.6,57.0,5.13,5.09,3.15,7.255,1,0,...,0,0,0,0,1,0,0,0,0,0
3,0.25,4,61.6,57.0,4.05,4.08,2.50,6.450,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0.52,2,62.0,55.0,5.16,5.19,3.21,7.721,0,0,...,0,0,0,0,0,0,0,1,0,0


In [110]:
X = train_df2.drop('price', axis=1)
y = train_df2.price

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [104]:
from sklearn.model_selection import GridSearchCV
model = RandomForestRegressor()
params = {
    "n_estimators":[600, 700], 
    "max_depth":[ None], 
    "min_samples_split":[8, 9, 10], 
    "min_samples_leaf":[1, 2]
}

In [105]:
df3_grid = GridSearchCV(estimator=model, param_grid=params, verbose=2, n_jobs=4, scoring= 'neg_mean_squared_error')

In [106]:
df3_grid.fit(X, y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=4,
             param_grid={'max_depth': [None], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [8, 9, 10],
                         'n_estimators': [600, 700]},
             scoring='neg_mean_squared_error', verbose=2)

In [107]:
df3_grid.best_params_

{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 8,
 'n_estimators': 700}

In [108]:
best = df3_grid.best_estimator_
best

RandomForestRegressor(min_samples_split=8, n_estimators=700)

In [109]:
y_pred_train = best.predict(X_train)
y_pred_test = best.predict(X_test)
mse_train = mean_squared_error(y_pred_train,y_train)
mse_test = mean_squared_error(y_pred_test,y_test)
mse_train, mse_test

(0.0035389528650617173, 0.0033545043562023197)

In [112]:
from sklearn.model_selection import GridSearchCV
model = RandomForestRegressor()
params = {
    "n_estimators":[700, 800, 1000], 
    "max_depth":[ 2, 5, None], 
    "min_samples_split":[8, 9, 10], 
    "min_samples_leaf":[1, 2]
}

In [113]:
df4_grid = GridSearchCV(estimator=model, param_grid=params, verbose=2, n_jobs=4, scoring= 'neg_mean_squared_error')

In [114]:
df4_grid.fit(X, y)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=4,
             param_grid={'max_depth': [2, 5, None], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [8, 9, 10],
                         'n_estimators': [700, 800, 1000]},
             scoring='neg_mean_squared_error', verbose=2)

In [115]:
df4_grid.best_params_

{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 8,
 'n_estimators': 700}

In [116]:
best = df4_grid.best_estimator_
best

RandomForestRegressor(min_samples_split=8, n_estimators=700)

In [117]:
y_pred_train = best.predict(X_train)
y_pred_test = best.predict(X_test)
mse_train = mean_squared_error(y_pred_train,y_train)
mse_test = mean_squared_error(y_pred_test,y_test)
mse_train, mse_test

(0.0034430228040152897, 0.0037342226089366946)

In [118]:
from sklearn.model_selection import GridSearchCV
model = RandomForestRegressor()
params = {
    "n_estimators":[600, 700], 
    "max_depth":[ None], 
    "min_samples_split":[7, 8], 
    "min_samples_leaf":[1, 2]
}

In [119]:
df5_grid = GridSearchCV(estimator=model, param_grid=params, verbose=2, n_jobs=4, scoring= 'neg_mean_squared_error')

In [120]:
df5_grid.fit(X, y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=4,
             param_grid={'max_depth': [None], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [7, 8],
                         'n_estimators': [600, 700]},
             scoring='neg_mean_squared_error', verbose=2)

In [121]:
df5_grid.best_params_

{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 7,
 'n_estimators': 700}

In [122]:
best = df5_grid.best_estimator_
best

RandomForestRegressor(min_samples_split=7, n_estimators=700)

In [123]:
y_pred_train = best.predict(X_train)
y_pred_test = best.predict(X_test)
mse_train = mean_squared_error(y_pred_train,y_train)
mse_test = mean_squared_error(y_pred_test,y_test)
mse_train, mse_test

(0.003118196474643327, 0.003372377128481128)

## changinig test

In [53]:
test_df

,id,carat,cut,depth,table,x,y,z,color_E,color_F,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0,0.51,4,58.3,61.9,5.19,5.20,3.04,1,0,...,0,0,0,0,0,1,0,0,0,0
1,1,1.02,4,63.0,58.0,6.37,6.43,4.03,0,1,...,0,0,0,0,0,1,0,0,0,0
2,2,0.59,2,61.9,56.0,5.39,5.34,3.32,0,0,...,0,0,0,0,0,0,0,1,0,0
3,3,0.90,4,62.3,56.0,6.14,6.18,3.84,0,0,...,0,0,0,0,0,0,0,1,0,0
4,4,2.01,5,60.2,61.0,8.23,8.16,4.93,0,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,1.05,5,61.9,58.0,6.50,6.45,4.01,0,0,...,1,0,0,0,1,0,0,0,0,0
13481,13481,0.32,2,62.1,57.0,4.35,4.38,2.71,0,1,...,0,0,0,0,0,0,0,1,0,0
13482,13482,0.41,5,61.3,59.0,4.78,4.74,2.92,0,0,...,0,0,1,0,0,0,1,0,0,0
13483,13483,0.30,5,62.2,59.0,4.31,4.28,2.67,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
encod = {
    "Fair":1,
    "Ideal":2,
    "Good":3,
    "Very Good":4,
    "Premium":5
}

In [33]:
test_df['cut'] = test_df['cut'].apply(lambda x: encod[x])

In [34]:
test_df = pd.get_dummies(test_df, columns=[ "color", "clarity"], drop_first=True)

In [56]:
test_df2= test_df.drop([ 'id', 'depth', 'table', 'z'], axis=1)

In [57]:
test_df2

,carat,cut,x,y,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.51,4,5.19,5.20,1,0,0,0,0,0,0,0,1,0,0,0,0
1,1.02,4,6.37,6.43,0,1,0,0,0,0,0,0,1,0,0,0,0
2,0.59,2,5.39,5.34,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0.90,4,6.14,6.18,0,0,0,0,0,0,0,0,0,0,1,0,0
4,2.01,5,8.23,8.16,0,0,0,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,1.05,5,6.50,6.45,0,0,0,1,0,0,0,1,0,0,0,0,0
13481,0.32,2,4.35,4.38,0,1,0,0,0,0,0,0,0,0,1,0,0
13482,0.41,5,4.78,4.74,0,0,0,0,0,1,0,0,0,1,0,0,0
13483,0.30,5,4.31,4.28,0,0,1,0,0,0,0,0,0,0,0,0,1


In [73]:
prediction = best.predict(test_df2)

In [74]:
prediction

array([[0.52271696, 4.        , 5.17615   , ..., 0.        , 0.        ,
        0.        ],
       [1.00299189, 4.        , 6.38213729, ..., 0.        , 0.        ,
        0.        ],
       [0.57091938, 2.        , 5.35425568, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.40731267, 5.        , 4.76744333, ..., 0.        , 0.        ,
        0.        ],
       [0.30316766, 5.        , 4.30326931, ..., 0.        , 0.        ,
        1.        ],
       [0.72026669, 2.        , 5.75926073, ..., 0.        , 0.        ,
        1.        ]])

In [70]:
subm_df = pd.DataFrame(prediction, columns = ['price'])
subm_df['id'] = subm_df.index

ValueError: Shape of passed values is (13485, 18), indices imply (13485, 1)

In [59]:
predicting = pd.DataFrame(test_df.id)

In [64]:
sub = pd.read_csv('submission_diamons.csv')

In [65]:
predicting = pd.DataFrame(sub.id)

In [ ]:
predicting